In [1]:
#http://alexminnaar.com/2019/08/22/ner-rnns-tensorflow.html
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

print(tf.__version__)

2.3.4


In [2]:
import pandas
import ast

pathTrain = "trains.csv"
pathTest = "tests.csv"
pathLabels = "labels.csv";

train_dataset = pandas.read_csv(pathTrain, index_col=0).applymap(ast.literal_eval).values.tolist()
test_dataset = pandas.read_csv(pathTest, index_col=0).applymap(ast.literal_eval).values.tolist()
labels = set(pandas.read_csv(pathLabels, index_col=0).values.flatten())

print(train_dataset[0:5])
print("Labels:", labels)

[[['i', 'need', 'a', 'place', 'to', 'dine', 'in', 'the', 'center', 'thats', 'expensive'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'restaurant-pricerange']], [['any', 'sort', 'of', 'food', 'would', 'be', 'fine', 'as', 'long', 'as', 'it', 'is', 'a', 'bit', 'expensive', '.', 'could', 'i', 'get', 'the', 'phone', 'number', 'for', 'your', 'recommendation', '?'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'restaurant-pricerange', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']], [['sounds', 'good', 'could', 'i', 'get', 'that', 'phone', 'number', 'also', 'could', 'you', 'recommend', 'me', 'an', 'expensive', 'hotel', '?'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'restaurant-pricerange', 'O', 'O']], [['yeah', 'i', 'need', 'a', 'restaurant', 'in', 'the', 'west', 'and', 'with', 'expensive', 'pricing', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'restaurant-area', 'O', 'O', 'restaurant-pricerange', 'O', 'O']], [['it', 's', '

In [3]:
# for i, label in enumerate(train_dataset[1]):
#     if 'other' in label:
#       train_dataset[1][i] = 'O'

# for i, label in enumerate(test_dataset[1]):
#     if 'other' in label:
#       test_dataset[1][i] = 'O'
      
# if 'other' in labels: labels.remove('other')
# labels.add('O')
# print(labels)

# create character vocab
all_text = " ".join([" ".join(x[0]) for x in train_dataset+test_dataset])
vocab = sorted(set(all_text))

# create character/id and label/id mapping
char2idx = {u:i+1 for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
label2idx = {u:i+1 for i, u in enumerate(labels)}
idx2label = np.array(labels)

print(idx2label)
print(char2idx)

{'restaurant-name', 'restaurant-booktime', 'O', 'restaurant-pricerange', 'restaurant-bookpeople', 'restaurant-food', 'restaurant-bookday', 'restaurant-area'}
{' ': 1, '!': 2, '#': 3, '%': 4, '&': 5, "'": 6, '*': 7, ',': 8, '-': 9, '.': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, '?': 22, '`': 23, 'a': 24, 'b': 25, 'c': 26, 'd': 27, 'e': 28, 'f': 29, 'g': 30, 'h': 31, 'i': 32, 'j': 33, 'k': 34, 'l': 35, 'm': 36, 'n': 37, 'o': 38, 'p': 39, 'q': 40, 'r': 41, 's': 42, 't': 43, 'u': 44, 'v': 45, 'w': 46, 'x': 47, 'y': 48, 'z': 49}


In [4]:
def split_char_labels(eg):
  '''
  For a given input/output example, break tokens into characters while keeping 
  the same label.
  '''

  tokens = eg[0]
  labels = eg[1]

  input_chars = []
  output_char_labels = []

  for token,label in zip(tokens,labels):
    input_chars.extend([char for char in token])
    input_chars.extend(' ')
    output_char_labels.extend([label]*len(token))
    output_char_labels.extend('O')

  return [[char2idx[x] for x in input_chars[:-1]], np.array([label2idx[x] for x in output_char_labels[:-1]])]


train_formatted = [split_char_labels(eg) for eg in train_dataset]
test_formatted = [split_char_labels(eg) for eg in test_dataset]

print(len(train_formatted))
print(len(test_formatted))

10193
1263


In [5]:
# training generator
def gen_train_series():

    for eg in train_formatted:
      yield eg[0],eg[1]

# validation generator
def gen_valid_series():

    for eg in valid_formatted:
      yield eg[0],eg[1]

# test generator
def gen_test_series():

  for eg in test_formatted:
      yield eg[0],eg[1]
  
# create Dataset objects for train, test and validation sets  
series = tf.data.Dataset.from_generator(gen_train_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
series_test = tf.data.Dataset.from_generator(gen_test_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))

BATCH_SIZE = 128
BUFFER_SIZE=1000

# create padded batch series objects for train, test and validation sets
ds_series_batch = series.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
ds_series_batch_test = series_test.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)

# print example batches
for input_example_batch, target_example_batch in ds_series_batch_test.take(1):
  print(input_example_batch)
  print(target_example_batch)

tf.Tensor(
[[31 28 35 ...  0  0  0]
 [26 24 37 ...  0  0  0]
 [24 26 43 ...  0  0  0]
 ...
 [32  1 46 ...  0  0  0]
 [26 24 37 ...  0  0  0]
 [43 41 48 ...  0  0  0]], shape=(128, 167), dtype=int32)
tf.Tensor(
[[3 3 3 ... 0 0 0]
 [3 3 3 ... 0 0 0]
 [3 3 3 ... 0 0 0]
 ...
 [3 3 3 ... 0 0 0]
 [3 3 3 ... 0 0 0]
 [3 3 3 ... 0 0 0]], shape=(128, 167), dtype=int32)


In [6]:
vocab_size = len(vocab)+1

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 256

label_size = len(labels)  

# build LSTM model
def build_model(vocab_size,label_size, embedding_dim, rnn_units, batch_size):
      model = tf.keras.Sequential([
          tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                    batch_input_shape=[batch_size, None],mask_zero=True),
          tf.keras.layers.LSTM(rnn_units,
                               return_sequences=True,
                               stateful=True,
                               recurrent_initializer='glorot_uniform'),
          tf.keras.layers.Dense(label_size)
          ])
      return model

model = build_model(
        vocab_size=len(vocab)+1,
        label_size=len(labels)+1,
        embedding_dim=embedding_dim,
        rnn_units=rnn_units,
        batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 256)          12800     
_________________________________________________________________
lstm (LSTM)                  (128, None, 256)          525312    
_________________________________________________________________
dense (Dense)                (128, None, 9)            2313      
Total params: 540,425
Trainable params: 540,425
Non-trainable params: 0
_________________________________________________________________


In [7]:
import os
# !pip install tensorflow-addons
# import tensorflow_addons as tfa
# import keras.backend as K
# from sklearn.metrics import f1_score

# def f1_metric(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     recall = true_positives / (possible_positives + K.epsilon())
#     f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
#     return f1_val


# define loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# def f1Score(labels, logits):
#   print(labels.numpy().shape, logits.numpy().shape)
#   return f1_score(labels.numpy(), logits.numpy())

# def recall(y_true, y_pred):
#     y_true = K.ones_like(y_true) 
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     all_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    
#     recall = true_positives / (all_positives + K.epsilon())
#     return recall

# def precision(y_true, y_pred):
#     y_true = K.ones_like(y_true) 
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def f1_score(y_true, y_pred):
#     precision_m = precision(y_true, y_pred)
#     recall_m = recall(y_true, y_pred)
#     return 2*((precision_m*recall_m)/(precision_m+recall_m+K.epsilon()))

# model.compile(optimizer='adam', loss=loss,metrics=[tf.keras.metrics.SparseCategoricalAccuracy(), f1_metric])
model.compile(optimizer='adam', loss=loss,metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [8]:
EPOCHS=45 # aparentemente melhor valor é 45, pq os valores não melhoram significativamente e o collab fica bem mais lento depois
history = model.fit(ds_series_batch, epochs=EPOCHS, validation_data=ds_series_batch_test,callbacks=[checkpoint_callback])
# pra o keras pegar o melhor: checkpoint_callback

Epoch 1/45
79/79 [==============================] - 73s 921ms/step - loss: 0.2945 - sparse_categorical_accuracy: 0.8370 - val_loss: 0.2319 - val_sparse_categorical_accuracy: 0.8493
Epoch 2/45
79/79 [==============================] - 74s 937ms/step - loss: 0.1835 - sparse_categorical_accuracy: 0.8698 - val_loss: 0.1492 - val_sparse_categorical_accuracy: 0.8881
Epoch 3/45
79/79 [==============================] - 69s 878ms/step - loss: 0.1267 - sparse_categorical_accuracy: 0.9012 - val_loss: 0.1151 - val_sparse_categorical_accuracy: 0.9080
Epoch 4/45
79/79 [==============================] - 72s 912ms/step - loss: 0.0997 - sparse_categorical_accuracy: 0.9202 - val_loss: 0.0958 - val_sparse_categorical_accuracy: 0.9240
Epoch 5/45
79/79 [==============================] - 75s 952ms/step - loss: 0.0814 - sparse_categorical_accuracy: 0.9346 - val_loss: 0.0793 - val_sparse_categorical_accuracy: 0.9367
Epoch 6/45
79/79 [==============================] - 85s 1s/step - loss: 0.0709 - sparse_categor

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

preds = np.array([])
y_trues= np.array([])

# iterate through test set, make predictions based on trained model
for input_example_batch, target_example_batch in ds_series_batch_test:

  pred=model.predict_on_batch(input_example_batch)
  pred_max=tf.argmax(tf.nn.softmax(pred),2).numpy().flatten()
  y_true=target_example_batch.numpy().flatten()

  preds=np.concatenate([preds,pred_max])
  y_trues=np.concatenate([y_trues,y_true])

# remove padding from evaluation
remove_padding = [(p,y) for p,y in zip(preds,y_trues) if y!=0]

r_p = [x[0] for x in remove_padding]
r_t = [x[1] for x in remove_padding]

# print confusion matrix and classification report
print(confusion_matrix(r_p,r_t))
print(classification_report(r_p,r_t, zero_division=0))

[[ 2263    14     1     0     1   106    11    60]
 [    7  1817     0     0     3   247     6    12]
 [    0     0   980     8     0   224     0     0]
 [    0     0    19   278     0    29     0     0]
 [    3     2     0     0  1892    95     0     2]
 [   35   425   165    15   172 68384    51   231]
 [   16    14     0     0     5   160  1441     1]
 [   25    19     2     0     3   209     0  2760]]
              precision    recall  f1-score   support

         1.0       0.96      0.92      0.94      2456
         2.0       0.79      0.87      0.83      2092
         3.0       0.84      0.81      0.82      1212
         4.0       0.92      0.85      0.89       326
         5.0       0.91      0.95      0.93      1994
         6.0       0.98      0.98      0.98     69478
         7.0       0.95      0.88      0.92      1637
         8.0       0.90      0.91      0.91      3018

    accuracy                           0.97     82213
   macro avg       0.91      0.90      0.90     8